In [1]:
import pandas as pd
import sqlite3
import zipfile
from pathlib import Path

In [2]:
data = pd.read_json('okved_2.json')
data.head()

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,


In [ ]:
conn = sqlite3.connect('db.db')
cur=conn.cursor()

### Задание №1

In [ ]:
#Создаем таблицу в БД

cur.execute("""
    create table if not exists okved(
        id integer primary key,
        code text,
        parent_code text,
        section text,
        name text,
        comment text);
""")
conn.commit()

In [ ]:
# В цикле проходимся по файлу и записываем данные в БД

for i in range(len(data)):
    cur.execute(
        """
            insert into okved(code, parent_code, section, name, comment)
            values (?,?,?,?,?);
        """,data.iloc[i]
        )
    conn.commit()

In [ ]:
#Проверка наличия записей в БД

cur.execute(
"""select * from okved
"""
)
res = cur.fetchall()
res

### Задание №2

In [ ]:
# Создаем таблицу

cur.execute("""
    create table if not exists okved_2(
        id integer primary key,
        name text,
        inn text,
        kpp text,
        okved text,
        full_name text);
""")
conn.commit()

In [ ]:
# Вспомогательная функция которая записываем в БД отфильтрованный датафрейм, содержащий только нужные строки
#  Так же проверяем перед записью наличие строк в БД с идентичным названием организации, если такая запись уже есть, то делаем
# ее апдейт.

def record_or_replace(df):
    for i in range(len(df)):
        name = df.iloc[i]['name'].replace("\"","'")
        inn = df.iloc[i]['inn']
        kpp = df.iloc[i]['kpp']
        okved_o = df.iloc[i]['data']['СвОКВЭД']['СвОКВЭДОсн']['КодОКВЭД']
        full_name = df.iloc[i]['full_name']
        cur.execute(
        "select * from okved_2 where name = \"" + name+"\""
        )
        res = cur.fetchall()
        if len(res)==0:
            cur.execute(
            """
                insert into okved_2(name,inn, kpp,okved,full_name)
                values (?,?,?,?,?);
            """,(name,str(inn),str(kpp),okved_o,full_name)
            )
        else:
            okved_o = okved_o+"; "+res[0][4]
            cur.execute(
            "update okved_2 set okved = \"" + okved_o +"\" where name = \"" + name+"\""
            )
    conn.commit()

In [ ]:
#Вспомогательная функция для фильтрации датафрейма

def okved(row):
    try:
        if(row['data']['СвОКВЭД']['СвОКВЭДОсн']['КодОКВЭД'][:2]=='61'):
            return "yes"
    except:
        return "no"
    return "no"

In [ ]:
#Читаем список файлов внутри архива

z = zipfile.ZipFile('egrul.json.zip')

In [ ]:
#Проходим по каждому файлу из списка, фильтруем его и результат записываем в БД

z.namelist()
for name in z.namelist():
    z.extract(name)
    with open(name,'r',encoding='utf-8') as f:
        data = pd.read_json(name)
        data['tst']=data.apply(okved,axis=1)
        dt2 = data[data.tst=='yes']
        record_or_replace(dt2)
    path = Path(name)
    path.unlink()

In [ ]:
cur.execute(
"""select * from okved_2

"""
)
res = cur.fetchall()